In [1]:
cd ../src

/Users/williamferreira/PycharmProjects/mscproject/src


In [2]:
import pandas as pd
import numpy as np

In [55]:
df_clean = pd.DataFrame.from_csv('../data/url-versions-2015-06-14-clean-test.csv')

In [56]:
# article headline stance ratios
df_clean.articleHeadlineStance.value_counts() / df_clean.articleHeadlineStance.value_counts().sum() * 100

for          48.722986
observing    33.005894
against      18.271120
dtype: float64

In [5]:
def claim_summary(df):
    # computes some summary stats for the given data set
    claim_grp = df.groupby('claimId').count().claimHeadline
    return claim_grp.min(), claim_grp.max(), claim_grp.mean(), claim_grp.std()

In [6]:
# summary stats for clean dataset
claim_summary(df_clean)

(1, 50, 8.65, 7.309738315937964)

In [7]:
from model.utils import get_tokenized_lemmas

In [8]:
overlap = []
for idx, row in df_clean.iterrows():
    claim_lemmas = get_tokenized_lemmas(row.claimHeadline)
    article_lemmas = get_tokenized_lemmas(row.articleHeadline)
    intersect = set(claim_lemmas).intersection(article_lemmas)
    union = set(claim_lemmas).union(article_lemmas)
    overlap.append(len(intersect) / float(len(union)))

In [9]:
df_clean['Overlap'] = overlap
df_clean['Overlap'] *= 100.0

In [14]:
thresholds = df_clean.groupby('articleHeadlineStance').Overlap.mean()

In [36]:
thresholds.sort()
thresholds

articleHeadlineStance
against      20.807913
observing    22.836092
for          24.297047
Name: Overlap, dtype: float64

In [59]:
min(thresholds.searchsorted(24.29), 2)

array([2])

In [62]:
thresholds.index[1]

'observing'

In [50]:
thresholds.index[min(thresholds.searchsorted(0)[0], 2)]

'against'

In [43]:
thresholds.index[2]

'for'

In [21]:
df_clean.groupby('articleHeadlineStance').Overlap.std()

articleHeadlineStance
against      12.272985
for          14.415870
observing    13.529762
Name: Overlap, dtype: float64

In [22]:
import numpy as np
import itertools as it

In [27]:
df_clean_train = pd.DataFrame.from_csv('../data/url-versions-2015-06-14-clean-train.csv')
df_clean_test = pd.DataFrame.from_csv('../data/url-versions-2015-06-14-clean-test.csv')

In [28]:
# claim summary stats for training data
claim_summary(df_clean_train)

(1, 40, 8.691666666666666, 7.093276719292539)

In [29]:
# claim summary stats for test data
claim_summary(df_clean_test)

(1, 50, 8.483333333333333, 8.181193122274697)

In [31]:
def stance_dist(df):
    # computes mean and count of stance labels in given df
    vc = df.articleHeadlineStance.value_counts() 
    return (vc / vc.sum() * 100, vc)

In [32]:
stance_dist(df_clean_test)[0]

for          48.722986
observing    33.005894
against      18.271120
dtype: float64

In [33]:
stance_dist(df_clean_test)[1]

for          248
observing    168
against       93
dtype: int64

In [34]:
stance_dist(df_clean_train)[0]

for          47.459252
observing    38.063279
against      14.477469
dtype: float64

In [35]:
stance_dist(df_clean_train)[1]

for          990
observing    794
against      302
dtype: int64